In [1]:
from pdf2image import convert_from_path
import pytesseract

# Path to the PDF
pdf_path = "Samples\Contract 3.pdf"


def extract_text_from_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image)
    return text


# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)
# print(pdf_text[:500])  # Print first 500 characters for verification

In [53]:
print(pdf_text)

HILABS CARE, INC.

ANCILLARY SERVICES AGREEMENT

This Ancillary Services Agreement (“Agreement”) is made and entered into as of the first day of January 2021, by and
between HILABS CARE, INC. doing business as BRAND NEW DAY (“Plan”) and ASCC LLC dba Riverwood
Healthcare Center (“Provider”), with reference to the following facts:

WHEREAS, Plan is a licensed health care services plan that has entered into or will enter into contracts with various
government and private Payors under which the Plan has agreed to provide or arrange health care services and benefits to
Plan Members under the Programs identified in Exhibit A hereto.

WHEREAS, Provider desires to participate in Plan’s network of contracting providers by providing Ancillary Services to
Members.

NOW THEREFORE, in consideration of the foregoing and for other good and valuable consideration, the parties hereto
agree as follows:

ARTICLE I.
DEFINITIONS

Whenever used in this Agreement, the following terms shall have the definitio

In [2]:
def chunk_text(text, chunk_size=1000):
    return [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]


# Chunk the extracted PDF text
chunks = chunk_text(pdf_text)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 11


In [4]:
from sentence_transformers import SentenceTransformer


# Initialize the MiniLM model

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def get_embeddings(model, texts):
    return model.encode(texts)



# Generate embeddings for the chunks
chunk_embeddings = get_embeddings(model, chunks)

print(f"Number of embeddings: {len(chunk_embeddings)}")

Number of embeddings: 11


In [56]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Initialize the HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Assuming 'chunks' is a list of text chunks
documents = [Document(page_content=chunk) for chunk in chunks]

# Create the Chroma collection
chroma_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="data",  # Specify where to save the collection
    collection_name="lc_chroma_demo",  # Choose a name for your collection
)

# Now let's search for relevant chunks based on a user query
user_query = "Provider has been approved to provide Ancillary Services under the Programs defined below and pursuant to the applicable terms and conditions of the Agreement, including, without limitation, Exhibits B through E hereto."
relevant_results = chroma_db.similarity_search(query=user_query, k=5)
print(relevant_results)

[Document(page_content='ider has been approved to provide Ancillary Services under the Programs defined below and pursuant to the\napplicable terms and conditions of the Agreement, including, without limitation, Exhibits B through E hereto.\n\nMedi-Cal Program: A state and federally funded Program pursuant to a contract between the Plan or Payor and DHCS\nfor coverage of Members who meet Medi-Cal eligibility requirements, as determined by DHCS.\n\nMedicare Program: A Program sponsored by Plan or Payor pursuant to a contract with the federal Centers for Medicare\nand Medicaid Services (“CMS Contract”) under the Medicare Advantage program.\n\nCommercial HMO Program: The commercial health benefit program(s) offered and sold by Plan to Subscriber\nGroups, including, without limitation, the Plan’s Health Benefit Off-Exchange Program.\nEXHIBIT E\n\nANCILLARY SERVICES\n\nSkilled Nursing Facility\n\n25\nEXHIBIT F\nCOMPENSATION\n\nThe following shall constitute 100% payment for services when co

In [57]:
print(relevant_results[0])

page_content='ider has been approved to provide Ancillary Services under the Programs defined below and pursuant to the
applicable terms and conditions of the Agreement, including, without limitation, Exhibits B through E hereto.

Medi-Cal Program: A state and federally funded Program pursuant to a contract between the Plan or Payor and DHCS
for coverage of Members who meet Medi-Cal eligibility requirements, as determined by DHCS.

Medicare Program: A Program sponsored by Plan or Payor pursuant to a contract with the federal Centers for Medicare
and Medicaid Services (“CMS Contract”) under the Medicare Advantage program.

Commercial HMO Program: The commercial health benefit program(s) offered and sold by Plan to Subscriber
Groups, including, without limitation, the Plan’s Health Benefit Off-Exchange Program.
EXHIBIT E

ANCILLARY SERVICES

Skilled Nursing Facility

25
EXHIBIT F
COMPENSATION

The following shall constitute 100% payment for services when combined with any required co-pay